In [1]:
#import bibliotek
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import json

In [4]:
url = 'https://www.worldathletics.org/records/all-time-toplists/sprints/100-metres/outdoor/men/senior?regionType=world&timing=electronic&windReading=regular&page=1&bestResultsOnly=true&firstDay=1896-04-06&lastDay=2022-11-04'

In [5]:
req = requests.get(url).content
soup = BeautifulSoup(req)

In [32]:
il_stron = int(str(soup.find('div', class_ = 'container--pagination__inner').find_all('form')[0]).split('max="')[1].split('"')[0])

In [214]:
#szukanie slownika skrot kraju - kraj
for i in range(len(soup.find_all('script'))):
    if 'requirejs' in str(soup.find_all('script')[i])[0:30]:
        slow = soup.find_all('script')[i]#.find(toplists.init()))

In [215]:
slow = str(slow).split('toplists.init(')[1][25:].split(',\n')[0]

In [216]:
#zamiana stringa na liste
slow_list = json.loads(slow)

In [220]:
#wyciagniecie slownika z listy
slow_slow = slow_list[5]['cases'][2]['values']
slow_slow

[{'value': 'afg', 'label': 'Afghanistan'},
 {'value': 'alb', 'label': 'Albania'},
 {'value': 'alg', 'label': 'Algeria'},
 {'value': 'asa', 'label': 'American Samoa'},
 {'value': 'and', 'label': 'Andorra'},
 {'value': 'ang', 'label': 'Angola'},
 {'value': 'aia', 'label': 'Anguilla'},
 {'value': 'ant', 'label': 'Antigua & Barbuda'},
 {'value': 'arg', 'label': 'Argentina'},
 {'value': 'arm', 'label': 'Armenia'},
 {'value': 'aru', 'label': 'Aruba'},
 {'value': 'aus', 'label': 'Australia'},
 {'value': 'aut', 'label': 'Austria'},
 {'value': 'aze', 'label': 'Azerbaijan'},
 {'value': 'bah', 'label': 'Bahamas'},
 {'value': 'brn', 'label': 'Bahrain'},
 {'value': 'ban', 'label': 'Bangladesh'},
 {'value': 'bar', 'label': 'Barbados'},
 {'value': 'blr', 'label': 'Belarus'},
 {'value': 'bel', 'label': 'Belgium'},
 {'value': 'biz', 'label': 'Belize'},
 {'value': 'ben', 'label': 'Benin'},
 {'value': 'ber', 'label': 'Bermuda'},
 {'value': 'bhu', 'label': 'Bhutan'},
 {'value': 'bol', 'label': 'Bolivia'},

In [537]:
slownik = {}
for i in range(len(slow_slow)):
    slownik[slow_slow[i]['value']] = slow_slow[i]['label']
slownik['gdr'] = 'Nrd'   #nrd
slownik['urs'] = 'Zsrr'  #zsrr - sprawdzic jak nrd i zsrr sa zapisane w tabelce z igrzysk
slownik['frg'] = 'Rfn'
slownik['aho'] = 'Netherlands Antilles'
slownik['tch'] = 'Czechoslovakia'
slownik['art'] = 'Athlete Refugee Team'
slownik['yug'] = 'Yugoslavia'
slownik['scg'] = 'Serbia and Montenegro'
slownik

{'afg': 'Afghanistan',
 'alb': 'Albania',
 'alg': 'Algeria',
 'asa': 'American Samoa',
 'and': 'Andorra',
 'ang': 'Angola',
 'aia': 'Anguilla',
 'ant': 'Antigua & Barbuda',
 'arg': 'Argentina',
 'arm': 'Armenia',
 'aru': 'Aruba',
 'aus': 'Australia',
 'aut': 'Austria',
 'aze': 'Azerbaijan',
 'bah': 'Bahamas',
 'brn': 'Bahrain',
 'ban': 'Bangladesh',
 'bar': 'Barbados',
 'blr': 'Belarus',
 'bel': 'Belgium',
 'biz': 'Belize',
 'ben': 'Benin',
 'ber': 'Bermuda',
 'bhu': 'Bhutan',
 'bol': 'Bolivia',
 'bih': 'Bosnia-Herzegovina',
 'bot': 'Botswana',
 'bra': 'Brazil',
 'ivb': 'British Virgin Islands',
 'bru': 'Brunei',
 'bul': 'Bulgaria',
 'bur': 'Burkina Faso',
 'bdi': 'Burundi',
 'cpv': 'Cabo Verde',
 'cam': 'Cambodia',
 'cmr': 'Cameroon',
 'can': 'Canada',
 'cay': 'Cayman Islands',
 'caf': 'Central African Republic',
 'cha': 'Chad',
 'chi': 'Chile',
 'tpe': 'Chinese Taipei',
 'col': 'Colombia',
 'dma': 'Commonwealth of Dominica',
 'com': 'Comoros',
 'cgo': 'Congo',
 'cok': 'Cook Islands',

In [420]:
soup_str = []
for i in range(il_stron):
    url_str = 'https://www.worldathletics.org/records/all-time-toplists/sprints/100-metres/outdoor/men/senior?regionType=world&timing=electronic&windReading=regular&page=' + str(i+1) + '&bestResultsOnly=true&firstDay=1896-04-06&lastDay=2022-11-04'
    req_str = requests.get(url_str).content
    soup_str.append(BeautifulSoup(req_str))

In [681]:
zawod = []
skrot_kr = []
kraj = []
numer = []
url_zawod = []
data = []
zawod_dane = {}

for k in range(len(soup_str)):
    ile_zawod = soup_str[k].find('table', class_ = 'records-table').find_all('tr')
    for i in range(1, len(ile_zawod)):
        url_zaw = 'https://www.worldathletics.org/athletes/'
        zaw = ile_zawod[i].find_all('a')[0].text.replace('  ', '').replace('\n','')
        zawod.append(zaw)

        ile_dan = ile_zawod[i].find_all('td')
        for j in range(len(ile_dan)):
            if str(ile_dan[j]).split('"')[1] == 'Nat':
                skrot = ile_dan[j].text.replace(' ', '').replace('\n','').lower()
                skrot_kr.append(skrot)
                kr = slownik[skrot]
                kraj.append(kr)

            if str(ile_dan[j]).split('"')[1] == 'Competitor':
                num = str(ile_dan[j].find_all('a')[0]).split('"')[1].split('-')[-1].split('=')[-1] 
                #print(i, num)                            #czasami jest =, a czasami - przed nr
                numer.append(num)

            if str(ile_dan[j]).split('"')[1] == 'DOB':
                if ile_dan[j].text == '\n':    #czasami nie ma daty urodzenia
                    data_urodz = 'None'
                else:
                    data_urodz = ile_dan[j].text.replace('  ', '').replace('\n', '') 
                data.append(data_urodz)
        url_z = url_zaw + kraj[k*100+i-1].replace(' ', '') + '/' + numer[k*100+i-1]
        url_zawod.append(url_z)
        zawod_dane[k*100+i-1] = {'nazwa': zaw, 'kraj': kr, 'skrot_kr': skrot, 'numer': num, 'data_urodz': data_urodz, 'url_zaw': url_z}

In [682]:
zawod_dane

{0: {'nazwa': 'Usain BOLT',
  'kraj': 'Jamaica',
  'skrot_kr': 'jam',
  'numer': '14201847',
  'data_urodz': '21 AUG 1986',
  'url_zaw': 'https://www.worldathletics.org/athletes/Jamaica/14201847'},
 1: {'nazwa': 'Tyson GAY',
  'kraj': 'United States',
  'skrot_kr': 'usa',
  'numer': '14238588',
  'data_urodz': '09 AUG 1982',
  'url_zaw': 'https://www.worldathletics.org/athletes/UnitedStates/14238588'},
 2: {'nazwa': 'Yohan BLAKE',
  'kraj': 'Jamaica',
  'skrot_kr': 'jam',
  'numer': '14201842',
  'data_urodz': '26 DEC 1989',
  'url_zaw': 'https://www.worldathletics.org/athletes/Jamaica/14201842'},
 3: {'nazwa': 'Asafa POWELL',
  'kraj': 'Jamaica',
  'skrot_kr': 'jam',
  'numer': '14202176',
  'data_urodz': '23 NOV 1982',
  'url_zaw': 'https://www.worldathletics.org/athletes/Jamaica/14202176'},
 4: {'nazwa': 'Justin GATLIN',
  'kraj': 'United States',
  'skrot_kr': 'usa',
  'numer': '14238562',
  'data_urodz': '10 FEB 1982',
  'url_zaw': 'https://www.worldathletics.org/athletes/UnitedSt

In [396]:
miesiace = {'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6, 'JUL': 7, 'AUG': 8, 'SEP':9, 'OCT': 10, 'NOV': 11, 'DEC': 12}

In [687]:
#dorobic date

dzien = []
mies = []
rok = []
for i in range(len(zawod_dane)):
    if zawod_dane[i]['data_urodz'].count(' ') == 2:                        #gdy poprawna data
        dzien.append(zawod_dane[i]['data_urodz'].split(' ')[0])
        mies.append(miesiace[zawod_dane[i]['data_urodz'].split(' ')[1]])
        rok.append(zawod_dane[i]['data_urodz'].split(' ')[2])
    elif zawod_dane[i]['data_urodz'] == 'None':
        dzien.append('None')
        mies.append('None')
        rok.append('None')
    elif zawod_dane[i]['data_urodz'].count(' ') == 0 and len(zawod_dane[i]['data_urodz']) == 4:   #tylko rok
        dzien.append('1')
        mies.append('1')
        rok.append(zawod_dane[i]['data_urodz'])
    elif zawod_dane[i]['data_urodz'].count(' ') == 1 and zawod_dane[i]['data_urodz'][:-5] in miesiace.keys() and zawod_dane[i]['data_urodz'][-4:].isnumeric():
        dzien.append('1')                                             #miesiac rok
        mies.append(miesiace[zawod_dane[i]['data_urodz'][:-5]])
        rok.append(zawod_dane[i]['data_urodz'][-4:])
    else:
        print(i, zawod_dane[i]['data_urodz'])

In [688]:
dat = []
for i in range(len(rok)):
    if rok[i] == 'None':
        dat.append('None')
    else:
        d = dzien[i] + '-' + str(mies[i]) + '-' + rok[i]
        dd = datetime.strptime(d, '%d-%m-%Y').date()
        dat.append(dd)
dat

[datetime.date(1986, 8, 21),
 datetime.date(1982, 8, 9),
 datetime.date(1989, 12, 26),
 datetime.date(1982, 11, 23),
 datetime.date(1982, 2, 10),
 datetime.date(1996, 3, 6),
 datetime.date(1995, 7, 10),
 datetime.date(1995, 5, 7),
 datetime.date(1996, 1, 2),
 datetime.date(1985, 10, 11),
 datetime.date(1974, 7, 23),
 datetime.date(1982, 11, 29),
 datetime.date(1994, 9, 26),
 datetime.date(1985, 6, 7),
 datetime.date(1989, 8, 29),
 datetime.date(1993, 10, 15),
 datetime.date(1967, 12, 16),
 datetime.date(1967, 7, 12),
 datetime.date(1993, 9, 21),
 datetime.date(1967, 2, 21),
 datetime.date(1984, 7, 9),
 datetime.date(1985, 4, 24),
 datetime.date(1993, 12, 15),
 datetime.date(1961, 7, 1),
 datetime.date(1967, 10, 2),
 datetime.date(1973, 12, 30),
 datetime.date(1978, 11, 22),
 datetime.date(1988, 3, 8),
 datetime.date(1992, 2, 27),
 datetime.date(1997, 7, 18),
 datetime.date(1996, 10, 7),
 datetime.date(1997, 12, 3),
 datetime.date(2001, 3, 16),
 datetime.date(2001, 11, 12),
 datetime.da

In [689]:
len(dat)

9220

In [690]:
df = pd.DataFrame(data = zawod_dane).transpose()
df

,nazwa,kraj,skrot_kr,numer,data_urodz,url_zaw
0,Usain BOLT,Jamaica,jam,14201847,21 AUG 1986,https://www.worldathletics.org/athletes/Jamaic...
1,Tyson GAY,United States,usa,14238588,09 AUG 1982,https://www.worldathletics.org/athletes/United...
2,Yohan BLAKE,Jamaica,jam,14201842,26 DEC 1989,https://www.worldathletics.org/athletes/Jamaic...
3,Asafa POWELL,Jamaica,jam,14202176,23 NOV 1982,https://www.worldathletics.org/athletes/Jamaic...
4,Justin GATLIN,United States,usa,14238562,10 FEB 1982,https://www.worldathletics.org/athletes/United...
...,...,...,...,...,...,...
9215,Ahmed Hossam EL DIN ALI ABU ZAID,Egypt,egy,14956182,29 JUL 2002,https://www.worldathletics.org/athletes/Egypt/...
9216,Akito SUZUKI,Japan,jpn,14951335,07 AUG 2003,https://www.worldathletics.org/athletes/Japan/...
9217,Toshiaki ATOMI,Japan,jpn,014963765,12 JAN 1997,https://www.worldathletics.org/athletes/Japan/...
9218,Jackson ROWE,Australia,aus,15006642,2003,https://www.worldathletics.org/athletes/Austra...


In [691]:
df['data_urodz'] = dat
df

,nazwa,kraj,skrot_kr,numer,data_urodz,url_zaw
0,Usain BOLT,Jamaica,jam,14201847,1986-08-21,https://www.worldathletics.org/athletes/Jamaic...
1,Tyson GAY,United States,usa,14238588,1982-08-09,https://www.worldathletics.org/athletes/United...
2,Yohan BLAKE,Jamaica,jam,14201842,1989-12-26,https://www.worldathletics.org/athletes/Jamaic...
3,Asafa POWELL,Jamaica,jam,14202176,1982-11-23,https://www.worldathletics.org/athletes/Jamaic...
4,Justin GATLIN,United States,usa,14238562,1982-02-10,https://www.worldathletics.org/athletes/United...
...,...,...,...,...,...,...
9215,Ahmed Hossam EL DIN ALI ABU ZAID,Egypt,egy,14956182,2002-07-29,https://www.worldathletics.org/athletes/Egypt/...
9216,Akito SUZUKI,Japan,jpn,14951335,2003-08-07,https://www.worldathletics.org/athletes/Japan/...
9217,Toshiaki ATOMI,Japan,jpn,014963765,1997-01-12,https://www.worldathletics.org/athletes/Japan/...
9218,Jackson ROWE,Australia,aus,15006642,2003-01-01,https://www.worldathletics.org/athletes/Austra...


In [738]:
soup_zaw = []
for i in range(len(df)):
    soup_zaw.append(BeautifulSoup(requests.get(df['url_zaw'][i]).content))

In [892]:
pb2 = []
for i in range(len(soup_zaw)):
    ss = soup_zaw[i].find_all('body')[0].find_all('script')[0].text.strip().split('resultsByYear.resultsByEvent.0.results.1')[0].split('"resultsByEvent"')
    if '100 Metres' not in str(ss):
        pb2.append('None')
    else:
        rekord = ss[0].split('100 Metres')[1].split('mark":"')[1].split('"')[0]
        if rekord[-1] == 'h':
            pb2.append(float(rekord[:-1]))
        else:
            pb2.append(float(rekord))

In [894]:
len(pb2)

9220

In [895]:
df['pb'] = pb2
df

,nazwa,kraj,skrot_kr,numer,data_urodz,url_zaw,pb
0,Usain BOLT,Jamaica,jam,14201847,1986-08-21,https://www.worldathletics.org/athletes/Jamaic...,9.58
1,Tyson GAY,United States,usa,14238588,1982-08-09,https://www.worldathletics.org/athletes/United...,9.69
2,Yohan BLAKE,Jamaica,jam,14201842,1989-12-26,https://www.worldathletics.org/athletes/Jamaic...,9.69
3,Asafa POWELL,Jamaica,jam,14202176,1982-11-23,https://www.worldathletics.org/athletes/Jamaic...,9.72
4,Justin GATLIN,United States,usa,14238562,1982-02-10,https://www.worldathletics.org/athletes/United...,9.74
...,...,...,...,...,...,...,...
9215,Ahmed Hossam EL DIN ALI ABU ZAID,Egypt,egy,14956182,2002-07-29,https://www.worldathletics.org/athletes/Egypt/...,10.55
9216,Akito SUZUKI,Japan,jpn,14951335,2003-08-07,https://www.worldathletics.org/athletes/Japan/...,10.55
9217,Toshiaki ATOMI,Japan,jpn,014963765,1997-01-12,https://www.worldathletics.org/athletes/Japan/...,10.55
9218,Jackson ROWE,Australia,aus,15006642,2003-01-01,https://www.worldathletics.org/athletes/Austra...,10.55


In [896]:
df.to_csv(r'C:\studia\informatyka_stosowana\semestr2\magisterka\df_iaaf_top.csv')


In [963]:
#soup_zaw[0]

In [ ]:
#pb na 100m  OK
#z io zrobic funkcje na kolor skory i dodac do df   OK
#sprobowac polaczyc ze soba df

#ogarnac pozostale dane z iaaf
#ogarnac sb, miejsce zawodow, date itp, wiek w chwili biegu